<a href="https://colab.research.google.com/github/noambassat/RAG_Agent_GITHUB_Rep.project/blob/main/ChromaDB_RAG_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:

!pip install -q pandas chromadb langchain sentence-transformers  --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 9.8 MB/s eta

In [14]:
# Imports
import pandas as pd

# Path to your file (replace with your actual path)
path = "/content/drive/MyDrive/GitHubRepositoriesProject/clean_df.xlsx"

# Load only relevant columns
df = pd.read_excel(path, usecols=["Name", "Description", "URL", "Topics"])

# Drop rows with missing Description or Topics
df.dropna(subset=["Description", "Topics"], inplace=True)

# Ensure Topics is a string
df["Topics"] = df["Topics"].astype(str)

# Combine into single text column
df["Full_Text"] = df["Description"] + " " + df["Topics"]

# Preview
df.head(2)


,Name,Description,URL,Topics,Full_Text
0,PyPOTS,toolboxlibrary data mining partially observed ...,https://github.com/WenjieDu/PyPOTS,"classification, clustering, data mining, forec...",toolboxlibrary data mining partially observed ...
1,changedetection.io,best simplest free open source website change ...,https://github.com/dgtlmoon/changedetection.io,"back in stock, change alert, change detection,...",best simplest free open source website change ...


In [15]:
df.shape

(11663, 5)

In [16]:
# Imports
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2") ## לבחור אימבדינגס מותאים!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# Generate embeddings
embeddings = embedding_model.encode(df["Full_Text"].tolist(), show_progress_bar=True)

# Convert to numpy array
embeddings = np.array(embeddings).astype("float32")

# Build FAISS index
dimension = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(embeddings)

# Save the index and dataframe (optional)
faiss.write_index(faiss_index, "faiss_index.index")
df.to_pickle("projects_df.pkl")


Batches:   0%|          | 0/365 [00:00<?, ?it/s]

In [18]:

import chromadb
from chromadb.config import Settings
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

# Re-initialize the embedding function
embedding_fn = SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

# Start Chroma client with in-memory DB
chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))

# Create collection
collection = chroma_client.create_collection(
    name="github_projects",
    embedding_function=embedding_fn
)

# Prepare documents for ChromaDB
documents = df["Full_Text"].tolist()
metadatas = df[["Name", "Description", "URL", "Topics"]].to_dict(orient="records")
ids = [str(i) for i in range(len(df))]

# Add to collection
collection.add(documents=documents, metadatas=metadatas, ids=ids)


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given


InternalError: ValueError: Batch size of 11663 is greater than max batch size of 5461